## RNN to Make a Model Learn Addition and Subtraction

## Model 1 : Many-to-Many RNN (parallel Processing)

This is a many-to-many sequence-to-sequence (seq2seq) model where the input and output sequences have <br>
the same length. Unlike the traditional encoder-decoder structure (where the encoder compresses an <br>
input into a single latent vector), this model passes the entire sequence through both RNNs.<br>


Given the string "54+7", the model should return a prediction: "61" <br>
Given the string "54-3", the model should return a prediction: "51" <br>

We will use an encoder-decoder architecture



### Load Libraries

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector, Input,LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.callbacks import Callback

from termcolor import colored

import sys
sys.stderr = open('err.txt', 'w')

2025-03-09 18:17:51.949013: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 18:17:51.956420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 18:17:51.965519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 18:17:51.968204: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 18:17:51.975145: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()
gpu_devices = [device for device in devices if device.device_type == 'GPU']
for gpu in gpu_devices:
    print('Using', gpu.physical_device_desc)

### Generate Data

In [3]:
all_chars    = '0123456789+-'
num_features = len(all_chars)
print('Number of features:', num_features)
trace = True
target_sequence_length = 5

Number of features: 12


In [4]:

char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))



In [5]:
def generate_data():
    first_num = np.random.randint(low=0,high=100)
    second_num = np.random.randint(low=0,high=100)
    add = np.squeeze(np.random.randint(low=0, high=100)) > 50.
    if add:
        example = str(first_num) + '+' + str(second_num)
        label = str(first_num+second_num)
    else:
        example = str(first_num) + '-' + str(second_num)
        label = str(first_num-second_num)
    return example, label

generate_data()

('80+55', '135')

### Support Functions

### Create the Model

In [6]:
"""

Interesting callback that prints the accuracy on each time step

"""
class SimpleLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Val Accuracy = {logs.get('val_accuracy', logs.get('val_acc', 'N/A')):.4f}")

simple_logger = SimpleLogger()

In [7]:
# Network Hyperparameters

hidden_units = 256
num_features = 12

model = Sequential([
    SimpleRNN(hidden_units, return_sequences=True, input_shape=(None, num_features)),  # Encoder RNN
    SimpleRNN(hidden_units, return_sequences=True),                                     # Decoder RNN
    Dense(num_features, activation='softmax')                                           # Final output layer
])


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, None, 256)      │       275,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 256)      │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 12)       │         3,084 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 803,852 (3.07 MB)

 Trainable params: 803,852 (3.07 MB)

 Non-trainable params: 0 (0.00 B)

### Vectorize and De-Vectorize Data

In [8]:
def vectorize_example(example, label):
    
    x = np.zeros((target_sequence_length, num_features))
    y = np.zeros((target_sequence_length, num_features))
    
    diff_x = target_sequence_length - len(example)
    diff_y = target_sequence_length - len(label)
    
    for i, c in enumerate(example):
        x[diff_x+i, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[diff_y+i, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
        
    return x, y

e, l = generate_data()
print('Text Example and Label:', e, l)
x, y = vectorize_example(e, l)
print('Vectorized Example and Label Shapes:', x.shape, y.shape)

Text Example and Label: 59-88 -29
Vectorized Example and Label Shapes: (5, 12) (5, 12)


In [9]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
    return ''.join(result)

def strip_padding(example):
    encountered_non_zero = False
    output = ''
    for c in example:
        if not encountered_non_zero and c == '0':
            continue
        if c == '+' or c == '-':
            encountered_non_zero = False
        else:
            encountered_non_zero = True
        output += c
    return output

devectorize_example(x)

'59-88'

In [10]:
print(devectorize_example(y), ':', strip_padding(devectorize_example(y)))

00-29 : -29


### Create Dataset

In [11]:
def create_dataset(num_examples=2000):

    x_train = np.zeros((num_examples, target_sequence_length, num_features))
    y_train = np.zeros((num_examples, target_sequence_length, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        x, y = vectorize_example(e, l)
        x_train[i] = x
        y_train[i] = y
    
    return x_train, y_train

x_train, y_train = create_dataset(20000)
print(x_train.shape, y_train.shape)

(20000, 5, 12) (20000, 5, 12)


In [12]:
devectorize_example(x_train[0])

'57+14'

In [13]:
devectorize_example(y_train[0])

'00071'

### Training the Model

In [14]:
def see_trained_output (preds):
    full_seq_acc = 0
    for i, pred in enumerate(preds):
        pred_str   = strip_padding(devectorize_example(pred))
        y_test_str = strip_padding(devectorize_example(y_test[i]))
        x_test_str = strip_padding(devectorize_example(x_test[i]))
        col = 'green' if pred_str == y_test_str else 'red'
        full_seq_acc += 1/len(preds) * int(pred_str == y_test_str)
        outstring = 'Input: {}, Out: {}, Pred: {}'.format(x_test_str, y_test_str, pred_str)
        print(colored(outstring, col))
    print('\nFull sequence accuracy: {:.3f} %'.format(100 * full_seq_acc))

In [15]:
""" 
    We train the model and see a random list of 20 examples how it is working 
    you will see the progression of the learning, but only if trace is True
"""
trace = True
for iteration in range(1,200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, epochs=1, validation_split=0.2, verbose=False,
         batch_size=1024, callbacks=[simple_logger])
    x_test, y_test = create_dataset(num_examples=20)
    preds = model.predict(x_test)
    if trace:
        see_trained_output(preds)


--------------------------------------------------
Iteration 1
Epoch 1: Loss = 2.3801, Val Accuracy = 0.5450
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Input: 16-84, Out: -68, Pred: 
Input: 31-25, Out: 6, Pred: 
Input: 55+81, Out: 136, Pred: 
Input: 25-26, Out: -1, Pred: 
Input: 74+92, Out: 166, Pred: 
Input: 69+73, Out: 142, Pred: 
Input: 15+32, Out: 47, Pred: 
Input: 85-33, Out: 52, Pred: 
Input: 36+52, Out: 88, Pred: 
Input: 78-96, Out: -18, Pred: 
Input: 80-28, Out: 52, Pred: 
Input: 92-44, Out: 48, Pred: 
Input: 22-43, Out: -21, Pred: 
Input: 44-16, Out: 28, Pred: 
Input: 24-6, Out: 18, Pred: 
Input: 13+19, Out: 32, Pred: 
Input: 23+77, Out: 100, Pred: 
Input: 48-79, Out: -31, Pred: 
Input: 79+61, Out: 140, Pred: 
Input: 57+53, Out: 110, Pred: 

Full sequence accuracy: 0.000 %

--------------------------------------------------
Iteration 2
Epoch 1: Loss = 1.9543, Val Accuracy = 0.5549
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Input: 34-4, Out: 30, Pred: 4
Input: 86+23, Out: 109, Pred: 15

Epoch 1: Loss = 1.0074, Val Accuracy = 0.6071
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Input: 15+84, Out: 99, Pred: 73
Input: 87-43, Out: 44, Pred: 13
Input: 92-20, Out: 72, Pred: 13
Input: 96-, Out: 96, Pred: 64
Input: 1+38, Out: 39, Pred: 94
Input: 28-83, Out: -55, Pred: -13
Input: 2+32, Out: 34, Pred: 94
Input: 52-18, Out: 34, Pred: -13
Input: 83+75, Out: 158, Pred: 111
Input: 23+72, Out: 95, Pred: 73
Input: 43-80, Out: -37, Pred: -13
Input: 42+98, Out: 140, Pred: 73
Input: 77+99, Out: 176, Pred: 111
Input: 44-89, Out: -45, Pred: -13
Input: 83+11, Out: 94, Pred: 113
Input: 24-87, Out: -63, Pred: -13
Input: 69+63, Out: 132, Pred: 111
Input: 3+2, Out: 5, Pred: 64
Input: 26-20, Out: 6, Pred: -13
Input: 90+44, Out: 134, Pred: 193

Full sequence accuracy: 0.000 %

--------------------------------------------------
Iteration 10
Epoch 1: Loss = 1.0019, Val Accuracy = 0.6034
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input: 35-14, Out: 21, Pred: -11
Input: 65+90, Out: 155, Pred: 118
Input: 50+10, O

Epoch 1: Loss = 0.9775, Val Accuracy = 0.6110
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input: 18+, Out: 18, Pred: 12
Input: 6-43, Out: -37, Pred: -42
Input: 41+96, Out: 137, Pred: 98
Input: 84+19, Out: 103, Pred: 154
Input: 64-83, Out: -19, Pred: 10
Input: 85+15, Out: 100, Pred: 154
Input: 19+39, Out: 58, Pred: 88
Input: 78-50, Out: 28, Pred: 11
Input: 54-10, Out: 44, Pred: 14
Input: 20+16, Out: 36, Pred: 88
Input: 37-94, Out: -57, Pred: -11
Input: 62+76, Out: 138, Pred: 113
Input: 37-93, Out: -56, Pred: -11
Input: 48-84, Out: -36, Pred: 10
Input: 99+32, Out: 131, Pred: 151
Input: 15+11, Out: 26, Pred: 84
Input: 71-25, Out: 46, Pred: 14
Input: 32-69, Out: -37, Pred: -14
Input: 44+14, Out: 58, Pred: 4
Input: 78+93, Out: 171, Pred: 154

Full sequence accuracy: 0.000 %

--------------------------------------------------
Iteration 18
Epoch 1: Loss = 0.9713, Val Accuracy = 0.6144
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input: 81-57, Out: 24, Pred: 24
Input: 56+86, Out: 142, Pred: 123
Input: 82-2

Epoch 1: Loss = 0.9173, Val Accuracy = 0.6341
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Input: 78+29, Out: 107, Pred: 164
Input: 56-57, Out: -1, Pred: 
Input: 70-37, Out: 33, Pred: 33
Input: 24+75, Out: 99, Pred: 68
Input: 28+81, Out: 109, Pred: 14
Input: 14-78, Out: -64, Pred: -53
Input: 74-34, Out: 40, Pred: 36
Input: 45+68, Out: 113, Pred: 18
Input: 93+30, Out: 123, Pred: 166
Input: 82-27, Out: 55, Pred: 66
Input: 57-16, Out: 41, Pred: 46
Input: 63+37, Out: 100, Pred: 138
Input: 42+41, Out: 83, Pred: 98
Input: 43-37, Out: 6, Pred: -23
Input: 51-30, Out: 21, Pred: 24
Input: 79+39, Out: 118, Pred: 144
Input: 42+67, Out: 109, Pred: 8
Input: 67+2, Out: 69, Pred: 73
Input: 41+91, Out: 132, Pred: 65
Input: 49+34, Out: 83, Pred: 198

Full sequence accuracy: 5.000 %

--------------------------------------------------
Iteration 26
Epoch 1: Loss = 0.9073, Val Accuracy = 0.6404
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Input: 48-58, Out: -10, Pred: 14
Input: 97+68, Out: 165, Pred: 144
Input: 43+26, Ou

Epoch 1: Loss = 0.8438, Val Accuracy = 0.6675
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Input: 73-51, Out: 22, Pred: 17
Input: 77-92, Out: -15, Pred: 56
Input: 41-86, Out: -45, Pred: -34
Input: 38+38, Out: 76, Pred: 77
Input: 94+30, Out: 124, Pred: 177
Input: 80+87, Out: 167, Pred: 166
Input: 97+48, Out: 145, Pred: 155
Input: 4+7, Out: 11, Pred: 12
Input: 31+54, Out: 85, Pred: 93
Input: 60+39, Out: 99, Pred: 183
Input: 10+77, Out: 87, Pred: 63
Input: 89-62, Out: 27, Pred: 27
Input: 70-19, Out: 51, Pred: 56
Input: 50+46, Out: 96, Pred: 97
Input: 61+24, Out: 85, Pred: 177
Input: 68-23, Out: 45, Pred: 36
Input: 45-3, Out: 42, Pred: 46
Input: 81-76, Out: 5, Pred: 27
Input: 14-9, Out: 5, Pred: 2
Input: 82+71, Out: 153, Pred: 151

Full sequence accuracy: 5.000 %

--------------------------------------------------
Iteration 34
Epoch 1: Loss = 0.8160, Val Accuracy = 0.6752
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Input: 16-15, Out: 1, Pred: --1
Input: 68-68, Out: , Pred: 17
Input: +16, Out: 16, Pred:

...

Epoch 1: Loss = 0.1855, Val Accuracy = 0.9020
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Input: 53-66, Out: -13, Pred: 13
Input: 56+79, Out: 135, Pred: 135
Input: 73-35, Out: 38, Pred: 38
Input: -90, Out: -90, Pred: -90
Input: 3+82, Out: 85, Pred: 85
Input: 11-51, Out: -40, Pred: -40
Input: 62+51, Out: 113, Pred: 113
Input: 69+85, Out: 154, Pred: 154
Input: 26-7, Out: 19, Pred: 29
Input: 41-9, Out: 32, Pred: 32
Input: 35-16, Out: 19, Pred: -19
Input: 22-47, Out: -25, Pred: -25
Input: 13-12, Out: 1, Pred: --1
Input: 73+94, Out: 167, Pred: 177
Input: 81-46, Out: 35, Pred: 35
Input: 16-21, Out: -5, Pred: --5
Input: 27+33, Out: 60, Pred: 60
Input: 93-18, Out: 75, Pred: 75
Input: 74-47, Out: 27, Pred: 37
Input: 87-44, Out: 43, Pred: 43

Full sequence accuracy: 65.000 %

--------------------------------------------------
Iteration 162
Epoch 1: Loss = 0.1847, Val Accuracy = 0.9036
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Input: 41-71, Out: -30, Pred: -30
Input: 82+38, Out: 120, Pred: 110
Input: 16+29

Epoch 1: Loss = 0.1835, Val Accuracy = 0.9000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Input: 85-58, Out: 27, Pred: 37
Input: 99-11, Out: 88, Pred: 88
Input: 3-59, Out: -56, Pred: -56
Input: 4-81, Out: -77, Pred: -87
Input: 59+59, Out: 118, Pred: 118
Input: 87-9, Out: 78, Pred: 88
Input: 23+92, Out: 115, Pred: 25
Input: 19-1, Out: 18, Pred: 18
Input: 74-33, Out: 41, Pred: 31
Input: 64+63, Out: 127, Pred: 127
Input: 90+12, Out: 102, Pred: 102
Input: 72+43, Out: 115, Pred: 115
Input: 67+75, Out: 142, Pred: 132
Input: 63+62, Out: 125, Pred: 125
Input: 85-56, Out: 29, Pred: 39
Input: 62+46, Out: 108, Pred: 108
Input: 94-89, Out: 5, Pred: 15
Input: 9+80, Out: 89, Pred: 99
Input: 27+88, Out: 115, Pred: 15
Input: 31-15, Out: 16, Pred: -16

Full sequence accuracy: 45.000 %

--------------------------------------------------
Iteration 170
Epoch 1: Loss = 0.1850, Val Accuracy = 0.9032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Input: 66+42, Out: 108, Pred: 108
Input: 68-52, Out: 16, Pred: 16
Input: 95+1

Epoch 1: Loss = 0.1831, Val Accuracy = 0.9028
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Input: 43+29, Out: 72, Pred: 162
Input: 12-29, Out: -17, Pred: -17
Input: 56+29, Out: 85, Pred: 185
Input: 81-81, Out: , Pred: -
Input: 55-2, Out: 53, Pred: 53
Input: 46-, Out: 46, Pred: 36
Input: 43-90, Out: -47, Pred: -57
Input: 60-21, Out: 39, Pred: 39
Input: 29+43, Out: 72, Pred: 72
Input: 81-23, Out: 58, Pred: 58
Input: 55+8, Out: 63, Pred: 63
Input: 9-96, Out: -87, Pred: -87
Input: 68-13, Out: 55, Pred: 55
Input: 21-39, Out: -18, Pred: -18
Input: 79+52, Out: 131, Pred: 131
Input: 34+49, Out: 83, Pred: 73
Input: 92+68, Out: 160, Pred: 150
Input: 1+5, Out: 6, Pred: 6
Input: 43+34, Out: 77, Pred: 177
Input: 33+21, Out: 54, Pred: 54

Full sequence accuracy: 60.000 %

--------------------------------------------------
Iteration 178
Epoch 1: Loss = 0.1836, Val Accuracy = 0.8980
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Input: 54+88, Out: 142, Pred: 132
Input: 62+37, Out: 99, Pred: 199
Input: 29-63, Out: -34

Epoch 1: Loss = 0.1823, Val Accuracy = 0.9009
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input: 81+45, Out: 126, Pred: 126
Input: 85-48, Out: 37, Pred: 47
Input: 56+79, Out: 135, Pred: 135
Input: 29+38, Out: 67, Pred: 67
Input: 26-97, Out: -71, Pred: -71
Input: 85-54, Out: 31, Pred: 31
Input: 82-95, Out: -13, Pred: 13
Input: 19-97, Out: -78, Pred: -78
Input: 81-20, Out: 61, Pred: 51
Input: 8-67, Out: -59, Pred: -59
Input: 39+39, Out: 78, Pred: 78
Input: 63+2, Out: 65, Pred: 65
Input: 13+36, Out: 49, Pred: 49
Input: 34+32, Out: 66, Pred: 76
Input: 59-79, Out: -20, Pred: 10
Input: 55-69, Out: -14, Pred: -4
Input: 94-63, Out: 31, Pred: 21
Input: 37+19, Out: 56, Pred: 56
Input: 1+86, Out: 87, Pred: 87
Input: 49+23, Out: 72, Pred: 172

Full sequence accuracy: 60.000 %

--------------------------------------------------
Iteration 186
Epoch 1: Loss = 0.1828, Val Accuracy = 0.8989
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Input: 69-71, Out: -2, Pred: -2
Input: 99-47, Out: 52, Pred: 52
Input: 46+14, Out

Epoch 1: Loss = 0.1831, Val Accuracy = 0.8993
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Input: 59-76, Out: -17, Pred: 17
Input: 41+1, Out: 42, Pred: 42
Input: 2+48, Out: 50, Pred: 40
Input: 77+94, Out: 171, Pred: 171
Input: 84-64, Out: 20, Pred: 20
Input: 85+8, Out: 93, Pred: 83
Input: 65-98, Out: -33, Pred: 23
Input: 1-21, Out: -20, Pred: -20
Input: 63+79, Out: 142, Pred: 132
Input: 16+1, Out: 17, Pred: 27
Input: 98+99, Out: 197, Pred: 187
Input: 78-40, Out: 38, Pred: 38
Input: 58-21, Out: 37, Pred: 37
Input: 5-76, Out: -71, Pred: -71
Input: 49-51, Out: -2, Pred: --2
Input: 82+19, Out: 101, Pred: 191
Input: 8+91, Out: 99, Pred: 9
Input: 31+56, Out: 87, Pred: 87
Input: 60+72, Out: 132, Pred: 132
Input: 54-49, Out: 5, Pred: -15

Full sequence accuracy: 45.000 %

--------------------------------------------------
Iteration 194
Epoch 1: Loss = 0.1822, Val Accuracy = 0.8988
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Input: 76-38, Out: 38, Pred: 48
Input: 3-68, Out: -65, Pred: -65
Input: 22-26, Out:

In [16]:
import session_info
session_info.show(html=False)

-----
keras               3.6.0
numpy               1.26.4
session_info        1.0.0
tensorflow          2.17.1
termcolor           NA
-----
IPython             8.28.0
jupyter_client      8.6.3
jupyter_core        5.7.2
-----
Python 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.39
-----
Session information updated at 2025-03-09 18:27
